DE dev notebook to debug script and set up to run on sherlock


#### create env

In [4]:
%%bash
cd /oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/scripts/taskfmri/temporal_integration

# python -m venv isc_env
# source isc_env/bin/activate
# python -m pip install --upgrade pip setuptools wheel
# pip install -r requirements.txt
# python -m ipykernel install --user --name isc_env --display-name "isc_env"
source /oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/scripts/taskfmri/temporal_integration/isc_env/bin/activate 

#### activate env

In [5]:
%%bash
source /oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/scripts/taskfmri/temporal_integration/isc_env/bin/activate /oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/scripts/taskfmri/temporal_integration/isc_env

#### install missing packages

In [ ]:
import os
def install(name):
    import subprocess
    subprocess.call(['pip', 'install',name])

install('config')
os.getcwd()

#### import packages

In [1]:
import os
import sys
import time
import subprocess




TI_CODE_DIR = "/oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/scripts/taskfmri/temporal_integration/TI_code"
if TI_CODE_DIR not in sys.path:
    sys.path.insert(0, TI_CODE_DIR)

from shared import config
print("CONFIG IMPORTED FROM:", config.__file__, flush=True)

from tqdm import tqdm


CONFIG IMPORTED FROM: /oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/scripts/taskfmri/temporal_integration/TI_code/shared/config.py


###### run from commandline

In [ ]:
# %cd /oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/scripts/taskfmri/temporal_integration/code/TI_code/isc

# %run run_isc_pipeline.py \
#   --condition TI1_orig \
#   --isc_method loo \
#   --stats_method bootstrap \
#   --n_perms 50 \
#   --p_threshold 0.05 \
#   --data_dir /oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/results/post_processed_wholebrain/filtered/06-2025/td/hpf \
#   --output_dir /oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/results/post_processed_wholebrain/filtered/06-2025/td/hpf/isc_analysis \
#   --mask_file /oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/scripts/taskfmri/temporal_integration/TI_code/mask/MNI152_T1_2mm_brain_mask.nii

In [2]:
import shared.pipeline_utils as pu
print("PIPELINE_UTILS IMPORTED FROM:", pu.__file__, flush=True)


PIPELINE_UTILS IMPORTED FROM: /oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/scripts/taskfmri/temporal_integration/TI_code/shared/pipeline_utils.py


In [4]:
cmd = [
    "python", "run_isc_pipeline.py",
    "--condition", "TI1_orig",
    "--isc_method", "loo",
    "--stats_method", "bootstrap",
    "--n_perms", "1000",
    "--p_threshold", "0.05",
    "--data_dir", "/oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/results/post_processed_wholebrain/filtered/06-2025/td/hpf",
    "--output_dir", "/oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/results/post_processed_wholebrain/filtered/06-2025/td/hpf/isc_analysis_1000_permutations",
    "--mask_file", "/oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/scripts/taskfmri/temporal_integration/TI_code/mask/MNI152_T1_2mm_brain_mask.nii"
]

p = subprocess.run(cmd, text=True)
print("Return code:", p.returncode)
print("\nSTDOUT:\n", p.stdout)
print("\nSTDERR:\n", p.stderr)


USING CONFIG: /oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/scripts/taskfmri/temporal_integration/TI_code/shared/config.py
Pipeline started at: Tue Jan 13 18:19:28 2026

=== STEP 1: ISC COMPUTATION ===
Running: /oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/scripts/taskfmri/temporal_integration/isc_env/bin/python /oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/scripts/taskfmri/temporal_integration/TI_code/isc/isc_compute.py --condition TI1_orig --method loo --data_dir /oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/results/post_processed_wholebrain/filtered/06-2025/td/hpf --output_dir /oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/results/post_processed_wholebrain/filtered/06-2025/td/hpf/isc_analysis_1000_permutations --mask_file /oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/scripts/taskfmri/temporal_integration/TI_code/mask/MNI152_T1_2mm_brain_mask.nii --chunk_size 3

## archive

In [ ]:

def run_command(cmd):
    print(f"\n[Pipeline] Running: {' '.join(cmd)}")
    result = subprocess.run(cmd, capture_output=False, text=True)
    if result.returncode != 0:
        print(f"Error running command: {cmd}")
        sys.exit(result.returncode)

def main(
    condition,
    isc_method="loo",
    stats_method="bootstrap",
    roi_id=None,
    n_perms=1000,
    threshold=0.05,
    data_dir=None,
    output_dir=None,
    mask_file=None,
):
    t_pipeline_start = time.time()

    if data_dir is None:
        data_dir = config.data_dir
    if output_dir is None:
        output_dir = config.output_dir
    if mask_file is None:
        mask_file = config.mask_file
    
    python_exec = sys.executable
    script_dir = os.path.dirname(os.path.abspath(__file__))
    
    
    # Path Arguments to pass down

    path_args = [
        "--data_dir", data_dir,
        "--output_dir", output_dir,
        "--mask_file", mask_file
    ]
    
    # 1. Run Step 1: ISC Computation
    print("\n[Pipeline] Step 1 starting (ISC computation)...")
    t_step1 = time.time()

    compute_script = os.path.join(script_dir, "isc_compute.py")
    cmd_step1 = [
        python_exec, "-u", compute_script,
        "--condition", condition,
        "--method", isc_method
    ] + path_args

    
    if roi_id is not None:
        cmd_step1.extend(['--roi_id', str(roi_id)])
        
    run_command(cmd_step1)
    
    # Define expected output file from Step 1 (needed for Step 2)
    # Naming convention in isc_compute.py:
    # isc_{condition}_{method}{roi_suffix}_desc-zscore.nii.gz
    roi_suffix = f"_roi{roi_id}" if roi_id is not None else ""
    output_dir = output_dir # Use arg instead of hardcode
    z_map_file = os.path.join(output_dir, f"isc_{condition}_{isc_method}{roi_suffix}_desc-zscore.nii.gz")
    
    print(f"[Pipeline] Step 1 finished in {(time.time() - t_step1)/60:.2f} min")

    # 2. Run Step 2: Statistics
    print("\n[Pipeline] Step 2 starting (Statistics)...")
    t_step2 = time.time()

    stats_script = os.path.join(script_dir, 'isc_stats.py')
    cmd_step2 = [
        python_exec, "-u", stats_script,
        '--method', stats_method,
        '--threshold', str(threshold),
        '--n_perms', str(n_perms)
    ] + path_args
    
    if roi_id is not None:
        cmd_step2.extend(['--roi_id', str(roi_id)])
        
    if stats_method == 'phaseshift':
        # Phase shift needs condition to reload data
        cmd_step2.extend(['--condition', condition])
    else:
        # Map-based methods need the input map
        cmd_step2.extend(['--input_map', z_map_file])
        
    run_command(cmd_step2)
    print(f"[Pipeline] Step 2 finished in {(time.time() - t_step2)/60:.2f} min")

    print("\n[Pipeline] Analysis Complete.")

    print(f"\n[Pipeline] Total runtime: {(time.time() - t_pipeline_start)/60:.2f} min")

def parse_args():
    parser = argparse.ArgumentParser(description="Run full ISC Analysis Pipeline")
    parser.add_argument("--condition", type=str, required=True,
                        help="Condition name (e.g., TI1_orig)")
    parser.add_argument("--isc_method", type=str, choices=["loo", "pairwise"], default="loo",
                        help="ISC method for computation (Step 1)")
    parser.add_argument("--stats_method", type=str, choices=["ttest", "bootstrap", "phaseshift"], default="bootstrap",
                        help="Statistical method for Step 2)")
    parser.add_argument("--roi_id", type=int, default=None,
                        help="Optional: ROI ID")
    parser.add_argument("--n_perms", type=int, default=1000,
                        help="Number of permutations bootstraps")
    parser.add_argument("--threshold", type=float, default=0.05,
                        help="P value threshold")

    parser.add_argument("--data_dir", type=str, default=config.DATA_DIR,
                        help=f"Path to input data (default: {config.DATA_DIR})")
    parser.add_argument("--output_dir", type=str, default=config.OUTPUT_DIR,
                        help=f"Output directory (default: {config.OUTPUT_DIR})")
    parser.add_argument("--mask_file", type=str, default=config.MASK_FILE,
                        help=f"Path to mask file (default: {config.MASK_FILE})")

    return parser.parse_args()


if __name__ == "__main__":
    args = parse_args()
    rc = main(
        condition=args.condition,
        isc_method=args.isc_method,
        stats_method=args.stats_method,
        roi_id=args.roi_id,
        n_perms=args.n_perms,
        threshold=args.threshold,
        data_dir=args.data_dir,
        output_dir=args.output_dir,
        mask_file=args.mask_file,
    )
    sys.exit(rc)

In [ ]:
import sys
import run_isc_pipeline

# Define parameters
data_dir = "/oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/results/post_processed_wholebrain/filtered/06-2025/td/hpf"
output_dir = "/oak/stanford/groups/menon/projects/daelsaid/2022_speaker_listener/results/post_processed_wholebrain/filtered/06-2025/td/hpf/isc_analysis"
mask_file = "/oak/stanford/groups/menon/projects/tongshan/2024_SL_TS/scripts/taskfmri/ISC_analysis/ISCtoolbox_v3_R340/templates/MNI152_T1_2mm_brain_mask.nii"

condition = "TI1_orig"
isc_method = "loo"
stats_method = "bootstrap"
n_perms = 50
threshold = 0.05
roi_id = None

# IMPORTANT: overwrite sys.argv BEFORE calling main()
sys.argv = [
    "run_isc_pipeline.py",
    "--condition", condition,
    "--isc_method", isc_method,
    "--stats_method", stats_method,
    "--n_perms", str(n_perms),
    "--threshold", str(threshold),
    "--data_dir", data_dir,
    "--output_dir", output_dir,
    "--mask_file", mask_file,
]

if roi_id is not None:
    sys.argv.extend(["--roi_id", str(roi_id)])

# Now call main
run_isc_pipeline.main()
